In [1]:
import os
import sys
import numpy as np
import cv2

from utils.homography import compute_h, cor_p
from utils.triangulation import triangulation
from pathlib import Path
from shapely.geometry import Polygon

In [2]:
num_cams = 4
cam_poses = {} # key: cami, value: pose
for i in range(num_cams):
    with open(f'./camera_poses/{i:05d}.txt', 'r') as f:
        lines = f.readlines()
        pose = []
        for line in lines:
            data = list(map(float, line.split(" ")))
            pose.append(data)
        pose = np.array(pose)
        cam_poses[f'cam{i}'] = pose.reshape(4, 4)

In [3]:
yolo_outputs = {} # key: cami, value: yolo_output

for i in range(num_cams):

    # outputs of one camera:
    yolo_output = {} # key: class, value: yolo_output
    yolo_output['chair'], yolo_output['table'], yolo_output['person'] = [], [], []

    with open(f'./runs/detect/layout/cam{i}/labels/00000.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            data = list(map(float, line.split(" ")))

            # check object class
            if data[0] == 56 or data[0] == 11: # chair or bench
                yolo_output['chair'].append(np.array(data[1:5]))
            elif data[0] == 60: # table
                yolo_output['table'].append(np.array(data[1:5]))
            elif data[0] == 0: # person
                yolo_output['person'].append(np.array(data[1:5]))
        yolo_outputs[f'cam{i}'] = yolo_output

In [10]:
h, w = 1456, 1928

poses = [cam_poses[f'cam{i}'][:3, :] for i in range(num_cams)]

K = np.array([
        [975.813843, 0, 960.973816],
        [0, 975.475220, 729.893921],
        [0, 0, 1]
    ])

for idx in range(len(cor_p)):
    X = [cor_p[i][idx][0] for i in range(num_cams)]
    Y = [cor_p[i][idx][1] for i in range(num_cams)]
    triangulated = triangulation(poses, X, Y)
    print(triangulated)
    for i in range(num_cams):
        reversed = K @ poses[i] @ np.array(triangulated.tolist() + [1])
        reversed = [reversed[0]/reversed[-1], reversed[1]/reversed[-1]]
        print(f"Reversed{i}: ", reversed)

[-2.64883981  0.29517656  4.03811247]
Reversed0:  [320.87907578806625, 801.1988747261788]
Reversed1:  [323.49884184275913, 530.2669335358781]
Reversed2:  [492.1333978947884, 463.53028434812904]
Reversed3:  [795.564651038152, 528.5538062304662]
[-1.66242026  1.44899178  3.67237594]
Reversed0:  [519.2399819301586, 1114.78250379963]
Reversed1:  [389.6405885546466, 842.1845146255065]
Reversed2:  [434.48941995892187, 714.3924150371314]
Reversed3:  [648.4333803761051, 692.5800242073246]
[-0.54477663 -0.27226253  2.8735826 ]
Reversed0:  [775.9780640706031, 637.4708428658666]
Reversed1:  [470.23949747407295, 425.37996819788884]
Reversed2:  [375.3493890658939, 383.4551564208737]
Reversed3:  [610.8301400963821, 425.5572811804795]
[ 10.60487483 -11.19737945 -12.22481217]
Reversed0:  [114.4672582801824, 1623.385455881854]
Reversed1:  [645.0150940229985, 1123.648282317398]
Reversed2:  [1046.4736398575624, 1099.1402347103951]
Reversed3:  [1423.3510385395766, 1497.0825922076078]
